In [5]:


from langsmith import Client

client = Client(api_url='https://eu.api.smith.langchain.com', api_key='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')

# Load the "search_council_alignment" dataset
dataset_name = "search_council_alignment"
dataset = client.read_dataset(dataset_name=dataset_name)

# Print some information about the dataset
print(f"Loaded dataset: {dataset.name}")
print(f"Number of examples: {dataset.example_count}")

# Optionally, you can list a few examples to verify the content
examples = list(client.list_examples(dataset_id=dataset.id, limit=5))
print("\nFirst 5 examples:")
for i, example in enumerate(examples, 1):
    print(f"\nExample {i}:")
    print(f"Inputs: {example.inputs}")
    print(f"Outputs: {example.outputs}")




Loaded dataset: search_council_alignment
Number of examples: 30

First 5 examples:

Example 1:
Inputs: {'input': 'Suche nach den Beschlüssen des Gemeinderats bezüglich der Drucksache G-24/065, die am 23.04.2024 behandelt wurde und sich auf die Zukunft der Freiburger Bauernmärkte bezog.'}
Outputs: {'output': 'Die Drucksache G-24/065 bezieht sich auf den interfraktionellen Antrag zur Sicherung der Zukunft der Freiburger Bauernmärkte, der am 07.10.2022 eingereicht wurde. Der Beschluss zu dieser Drucksache wurde bereits zugestellt. Weitere Details zur Behandlung am 23.04.2024 sind in dem bereitgestellten Kontext jedoch nicht enthalten. \n\nQuelle: Unbekannt.'}

Example 2:
Inputs: {'input': 'Was war der Beschluss des Gemeinderats bezüglich der Drucksache G-24/065 zur Zukunft der Freiburger Bauernmärkte?'}
Outputs: {'output': 'Der Beschluss des Gemeinderats bezüglich der Drucksache G-24/065 zur Zukunft der Freiburger Bauernmärkte ist nicht direkt im bereitgestellten Kontext angegeben. Es wir

In [16]:
# Extract inputs from examples
inputs = []
for example in examples:
    input_text = example.inputs.get('input', '')
    if input_text:
        inputs.append(input_text)

# Print the list of inputs
print("\nList of inputs:")
for i, input_text in enumerate(inputs, 1):
    print(f"{i}. {input_text}")


List of inputs:
1. Suche nach den Beschlüssen des Gemeinderats bezüglich der Drucksache G-24/065, die am 23.04.2024 behandelt wurde und sich auf die Zukunft der Freiburger Bauernmärkte bezog.
2. Was war der Beschluss des Gemeinderats bezüglich der Drucksache G-24/065 zur Zukunft der Freiburger Bauernmärkte?
3. Was wurde mit dem Dokument G-24/065 in Bezug auf die Entscheidungen zu den Freiburger Bauernmärkten beschlossen?
4. Was wurden die Entscheidungen des Gemeinderats bezüglich der Freiburger Bauernmärkte?
5. Suche nach dem Datum der letzten Sitzung des Freiburger Gemeinderats, in der das Eingemeindungsjubiläum einer Ortschaft gewürdigt wurde.


In [11]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

True

In [12]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [30]:
parent_query = """
MATCH (node)-[:FILE_SECTION_HAS_FILE_CHUNK]-(parent)
MATCH (parent)-[:FILE_HAS_FILE_SECTION]-(file)
WITH file, parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {source: file.access_url} AS metadata LIMIT 1
"""

In [31]:
typical_rag = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(), index_name="file_chunks",retrieval_query=parent_query
)

In [32]:
retriever = typical_rag.as_retriever(search_kwargs={"k": 10})

for input_text in inputs:
    print(input_text)
    docs = retriever.get_relevant_documents(input_text)
    print(f"\nRelevant documents for: '{input_text}'\n")
    for i, doc in enumerate(docs, 1):
        print(f"Document {i}:")
        print(f"Content: {doc.page_content[:200]}...")  # Print first 200 characters
        print(f"Metadata: {doc.metadata}")
        print()
    print("-" * 100)

Suche nach den Beschlüssen des Gemeinderats bezüglich der Drucksache G-24/065, die am 23.04.2024 behandelt wurde und sich auf die Zukunft der Freiburger Bauernmärkte bezog.

Relevant documents for: 'Suche nach den Beschlüssen des Gemeinderats bezüglich der Drucksache G-24/065, die am 23.04.2024 behandelt wurde und sich auf die Zukunft der Freiburger Bauernmärkte bezog.'

Document 1:
Content: 
   (Drucksache G-24/044 bereits zugestellt; Drucksache G-24/044.1 wird nachgereicht)

8. Freiburger Sportentwicklungsplan 2024 - 2040 (u. a. Antrag der CDU-Fraktion vom 14.02.2023)
...
Metadata: {'source': 'https://ris.freiburg.de/documents.php?id=69&inline=1&document_type_id=11&meeting_attachment_id=ni_2024-GR-262%7C20240308090002-0_2024-GR-262_1.pdf'}

----------------------------------------------------------------------------------------------------
Was war der Beschluss des Gemeinderats bezüglich der Drucksache G-24/065 zur Zukunft der Freiburger Bauernmärkte?

Relevant documents for: 'Was wa

In [38]:
from typing import List, Union

from langchain.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema
from langchain_community.graphs import Neo4jGraph
from langchain_core.messages import (
    AIMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Access environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# Connection to Neo4j
graph = Neo4jGraph()

In [39]:
from langchain_aws import ChatBedrock

cypher_llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0),
    name="agent",
    # other params...
)

In [49]:
cypher_template = (
    cypher_template
) = """
Create a Cypher query to filter embedding search results on file chunks in a Neo4j graph database. Start with 'node' (unlabeled nodes from initial search). Your query should:

1. Traverse relationships to relevant connected nodes
2. Filter using only structural properties and relationships (no text content)
3. Include the file's access URL as 'source' in the output metadata

Key points:
- NO text-based filtering (no CONTAINS, =, etc. on string fields)
- Use only relationships and non-text properties (dates, numbers, booleans)
- Always reach and return the File node's access_url as 'source'
- Include score in the query

Key node types: File, FileSection, FileChunk, Meeting, AgendaItem, Paper, Person, Organization

Query structure:
MATCH (node)-[:<RELATIONSHIPS>]->(:RelevantNodes)
MATCH (file:File) WHERE ...
WHERE <structural_conditions>
WITH node, score, file
RETURN node.id, score, node.text, {{source: file.access_url, ...}} AS metadata

- Include all properties in the WITH clause that are used in the RETURN statement

If there is a unique identifier like a reference present in the question, then use that and nothing else should get the document directly.

e.g. 

Suche nach den Beschlüssen des Gemeinderats bezüglich der Drucksache G-24/065, die am 23.04.2024 behandelt wurde und sich auf die Zukunft der Freiburger Bauernmärkte bezog.

returns:

MATCH (node)-[:FILE_SECTION_HAS_FILE_CHUNK]-(fileSection:FileSection)
)
MATCH (file:File) - [:FILE_HAS_FILE_SECTION]->(fileSection:FileSection)
MATCH(paper:Paper)-[:PAPER_HAS_AUXILIARY_FILE]->(file:File)
WHERE paper.reference = 'G-24/065'
RETURN node.id, score, node.text, {{source: file.access_url, ...}} AS metadata

Schema: {schema}
Question: {question}
Cypher query:
"""


In [50]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to an unlabelled Cypher filter query. No pre-amble. No ''' or ```",
        ),
        ("human", cypher_template),
    ]
)

cypher_response = (
    RunnablePassthrough.assign(
        schema=lambda _: graph.get_schema,
    )
    | cypher_prompt
    | cypher_llm.bind(stop=["\nCypherResult:"])
    | StrOutputParser()
)

In [54]:
question = "Suche nach den Beschlüssen des Gemeinderats bezüglich der Drucksache G-24/065, die am 23.04.2024 behandelt wurde und sich auf die Zukunft der Freiburger Bauernmärkte bezog."

In [55]:
response = cypher_response.invoke({"question":question})
print(response)

MATCH (node)-[:FILE_HAS_FILE_SECTION|FILE_SECTION_HAS_FILE_CHUNK*]->(chunk:FileChunk)
MATCH (paper:Paper)-[:PAPER_HAS_AUXILIARY_FILE]->(file:File)
WHERE paper.reference = 'G-24/065'
WITH node, score, file, paper
MATCH (paper)-[:PAPER_HAS_CONSULTATION]->(consultation:Consultation)
MATCH (consultation)-[:CONSULTATION_HAS_MEETING]->(meeting:Meeting)
WHERE date(meeting.start) = date('2024-04-23')
RETURN node.id, score, node.text, {source: file.access_url, paperReference: paper.reference, meetingDate: meeting.start} AS metadata


In [56]:
smart_rag = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(), index_name="file_chunks",retrieval_query=response
)

In [57]:
retriever = smart_rag.as_retriever(search_kwargs={"k": 10})
docs = retriever.get_relevant_documents("What is the name of the person who is the head of the organization?")
print(docs)

[]
